# Fusion

Submission 2:
    - Tremor : Gradient Boosting Regression
    - Dyskinesia : Gradient Boosting Regression
    
Submission 4: 
    - Dyskinesia : Average of predictions 

In [6]:
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
import csv
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from random import random
from sklearn.metrics import mean_squared_error
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

%load_ext autoreload
%autoreload 2

from transform_data import *
from create_graphs import *
from local.get_final_scores_accuracy import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
sDirOut = "/export/b19/mpgill/BeatPD_predictions_tryingout"

## Submission 2 - Gradient Boosting Regression

In [43]:
def read_files(lFilesPred, sFileLabels, sTypeLabel):
    """
    TODO 
    
    Keyword Arguments: 
    - lFilesPred: 
    - sFileLabels: 
    - sTypeLabels: 
    """
    
    # Read file labels (true labels)
    print("sFileLabels : ", sFileLabels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
    
    # Training-testing data
    iNumFiles=len(lFilesPred)
    #mPredictions=[] #np.zeros((1,iNumFiles))
    #mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    vLabels=[] #true label
    vParID=[] # participant ID
    lDicts=[] 


    # We loop over the predictions files from the tsfresh and SVR
    # lDicts is a list of measurement_id and the predictions obtained 
    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
    print('sTypeLabel : ', sTypeLabel)
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
    
    print('dLabels')
    print(dLabels)
    
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            if dLabels[k]!='NA':
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
                vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
                for j in range(1, iNumFiles):
                    fPred=lDicts[j].get(k)
                if fPred:
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
                    
            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    print('mPredictions and vLabels')
    print(mPredictions.shape)
    print(len(vLabels))
    # Random forest training - regression
    
    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())
    return mPredictions, vLabels

In [7]:
def RandomForest_fusion(lFilesPred, sFileLabels, iEstimators, sTypeLabel, nudge, rLR, iMD,iRS):

    mPredictions, vLabels = read_files(lFilesPred, sFileLabels, sTypeLabel)
    
    clf=GradientBoostingRegressor(n_estimators=iEstimators, learning_rate=rLR, max_depth=iMD,\
                                      random_state=iRS, loss='ls').fit(mPredictions, vLabels)
    vNewPred=clf.predict(mPredictions)
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]
    
    # we will include the testing data here
    return (vNewPred, vParID, vLabels, vRes1, vRes2, clf)
#print(mPredictions)

In [8]:
def randomForest_predTest(lTestPred, clf, sDirOut,sName):
    
    lDicts=[] #Will contain the two test predictions
    lmesID=[]
    iNumFiles=len(lTestPred)
    vPredIter=np.zeros((1,iNumFiles))

    for sFilePred in lTestPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
      
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            lmesID.append(k)
            #vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            for j in range(1, iNumFiles):
                fPred=lDicts[j].get(k)
            if fPred:
                vPredIter[0,j]=float(np.asarray(fPred))
            else:
                print(['Unkwnown key:' + k])
                vPredIter[0,j]=float(np.asarray(lDicts[0][k]))

            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    
    vTestPred=clf.predict(mPredictions)
    df = pd.DataFrame({'measurement_id': lmesID, sName:vTestPred})
    df.to_csv(sDirOut+'submissionCisPD'+sName+'.csv', index=False)
    return(lmesID, vTestPred)



### Dyskinesia

In [8]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_on_off.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sFilePred2='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTypeLabel='on_off'
n_estimators=300
rLR=0.1
iMD=1
iRS=15
nudge=100
sFileLabels=sFilePred1

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1,\
vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel, nudge, rLR, iMD,iRS)

print('Overall MSE Classif. 1 : ', get_final_score(vTrueLabels,np.array(vParID).astype(int), vRes1))
print('Overall MSE Classif. 2 : ', get_final_score(vTrueLabels, vParID, vRes2))
print('Overall MSE Fusion : ', get_final_score(vPredCross, vParIDCross,vTrueCross))




sTestPred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/On_off_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/'
sName='on_off'
#lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
#print(vTestPred)




IndexError: list index out of range

### Tremor - Submission 2

In [5]:
# tsfresh predictions file 
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_tremor.csv'

# SVR predictions file 
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold_all/objs_450_kernel_linear_c_0.02_eps_0.1.csv'

# subchallenge 
sTypeLabel='tremor'

# RandomForest hyperparameter 
n_estimators=5

# Labels
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'


lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion(lFilesPred,
                                                                           sFileLabels,
                                                                           n_estimators,
                                                                           sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.tremor.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/Tremor_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='tremor'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



NameError: name 'RandomForest_fusion' is not defined

### Dyskinesia - Submission 2

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_dyskinesia.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'
sTypeLabel='dyskinesia'
n_estimators=5
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold/Dyskinesia_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='z_dyskinesia'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



## Submission 4 - Average of predictions 

In [110]:
def read_files_average(lFilesPred, sFileLabels, sTypeLabel):
    """
    TODO 
    
    Keyword Arguments: 
    - lFilesPred: 
    - sFileLabels: 
    - sTypeLabels: 
    """
    
    # Read file labels (true labels)
    print("sFileLabels : ", sFileLabels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
    
    # Training-testing data
    iNumFiles=len(lFilesPred)
    #mPredictions=[] #np.zeros((1,iNumFiles))
    #mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    vLabels=[] #true label
    vParID=[] # participant ID
    lDicts=[] 


    # We loop over the predictions files from the tsfresh and SVR
    # lDicts is a list of measurement_id and the predictions obtained 
    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
    print('sTypeLabel : ', sTypeLabel)
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
    
#     print(lDicts[0])
#     print('-------------------------')
#     print(lDicts[0].keys() == lDicts[1].keys())
    
#     print('Aaaaaaaaaaaaaaaaaaaaaaaa')
#     print(lDicts[0].keys())
#     print('Aaaaaaaaaaaaaaaaaaaaaaaa')
#     print(lDicts[1].keys())
    
    print(lDicts[0].keys())
    
    #creation of the matrix containing prediction from all classifiers
#     bEnter=1
#     dMeasurements=[]
#     for k in lDicts[0]: #first dictionary will be the lead
#         if k!='measurement_id':
#             print('k !!!: ', k)
#             #print(dID[k])
#             if dLabels[k]!='NA':
#                 vLabels.append(float(np.asarray(dLabels[k]))) #true labels
#                 vParID.append(float(np.asarray(dID[k]))) #participant ID
#                 dMeasurements.append(k)
#                 print('First predicted value : ', float(np.asarray(lDicts[0][k])))
#                 vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
#                 for j in range(1, iNumFiles):
#                     fPred=lDicts[j].get(k)
#                 if fPred:
#                     vPredIter[0,j]=float(np.asarray(fPred))
#                 else:
#                     print(['Unkwnown key:' + k])
#                     vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
#             if bEnter==1:
#                 mPredictions=vPredIter
#                 bEnter=0
#             else:
#                 mPredictions=np.append(mPredictions,vPredIter,axis=0)
#     print(len(dMeasurements))
    lmesID=[]
    
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            lmesID.append(k)
            #vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            for j in range(1, iNumFiles):
                fPred=lDicts[j].get(k)
            if fPred:
                vPredIter[0,j]=float(np.asarray(fPred))
            else:
                print(['Unkwnown key:' + k])
                vPredIter[0,j]=float(np.asarray(lDicts[0][k]))

            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
    
    # Adding the measurement_id columns. It is the same order for both lDicts[0] and lDicts[1]
#     if lDicts[0].keys() == lDicts[1].keys() :
#         mPredictions = np.concatenate((lDicts[0].keys(), mPredictions), axis=1)
    print('mPredictions and vLabels')
    print(mPredictions.shape)
    print(len(vLabels))
    # Random forest training - regression
    
    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())
    return mPredictions, vLabels

In [113]:
def average_fusion(lFilesPred, dest_dir, fileName, bRound):
    
    # Read file labels (true labels)
    
    iNumFiles=len(lFilesPred)
    print('Number of analyzed file inputs: '+ str(iNumFiles))
#mPredictions=[] #np.zeros((1,iNumFiles))
#mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    lID=[] #true label
    lDicts=[] 

    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (acoustic, w-embed)
        lDicts.append(dPred)
   
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            lID.append(k)
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            #print(vPredIter)
            for j in range(1, iNumFiles):
                fPred=lDicts[j].get(k)
                if fPred:
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))

            if bEnter==1:
                mPredictions=vPredIter # mPredictions initialization.
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)

    print('mPredictions and vID')
    print(mPredictions.shape)
    print(len(lID))
    print(mPredictions)
    vAverage=np.mean(mPredictions,axis=1)
    if bRound==1:
        vPrediction=np.round(vAverage)
    else:
        vPrediction=vAverage
    
    
          
    # Random forest training - regression
    
    
#clf=RandomForestClassifier(n_estimators)
#clf = clf.fit(X, Y)
#clf.score(X_test, y_test)

    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())
    lID, vPrediction=zip(*sorted(zip(lID, vPrediction)))
    df = pd.DataFrame({'ID': lID, 'Prediction':vPrediction})
    np.savetxt(dest_dir+fileName+'.txt', df.values, fmt='%s', delimiter=' ; ')
   
   
    # we will include the testing data here
    #print(mPredictions)
    
    return vAverage, lID

In [111]:
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'
sTypeLabel = "dyskinesia"
lFilesPred=[sFilePred1,sFilePred2]

# mPredictions contains in one column the ivector prediction, and the other column is the tsfresh predictions 
mPredictions, vLabels = read_files_average(lFilesPred, sFileLabels, sTypeLabel)

meanPredictions = np.mean(mPredictions, axis=1)

print(mPredictions)
# Retrieve the measurements id? 
print(meanPredictions)

# Compare these results with the 4th submission 



sFileLabels :  /export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv
sTypeLabel :  dyskinesia
dict_keys(['measurement_id', '3f8579af-2b1b-4fc9-8a5e-5310dd352b5d', '4bff3454-c6dd-4264-bd07-a4c2b537a0f5', '4ef75ae1-08dc-4d63-bae5-880e0341b86f', '5d4da060-f459-4a1b-8225-ef4f3bfc0862', '6ab48002-f9af-4fc7-a433-8d89b1ec45a5', 'e6434482-4e09-4021-a256-1fe7aa0768c9', 'eb08e5ab-506c-41bb-b791-dc2430616449', '2bf17d8f-53c0-4d61-957d-2495f26228a5', '93eac078-c686-4532-89e7-b0cd17eae12e', '9bfa377e-074a-45bc-afd4-6aaa4f91fede', 'e32bbd29-6933-4bdd-9d01-d84ced190a64', 'a44de75f-ed52-4e46-b01f-3a9b59e47276', 'b4940eff-ab6b-4f85-9490-c8c3d9869623', 'b51d22e9-3287-4ccd-b312-47196175f536', 'bc339c17-0f1a-4e79-b68e-1b6884a32814', 'ceb68e3f-9905-449f-9deb-824ffbe4c079', 'cfa9ec65-2dfa-43ad-955f-2fb05673c795', 'd2d1b435-029c-4a36-bceb-1f1bcd4c1dec', 'df8f969b-5f9b-49e3-9ef8-883776e3bbf1', '070aa678-8343-48f7-9e89-c7de38c0e5c6', 'f24273b8-eb74-4043-8f08-d7623c0d2c2b'

In [117]:
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

dest_dir="/export/b19/mpgill/BeatPD_predictions_tryingout/"
fileName ="dysk_test"

sTypeLabel = "dyskinesia"

lFilesPred=[sFilePred1,sFilePred2]
bRound=0
vAverage, lID = average_fusion(lFilesPred, dest_dir, fileName, bRound)

print(vAverage)
print(lID)

Number of analyzed file inputs: 2
mPredictions and vID
(1188, 2)
1188
[[1.10496896 0.98700742]
 [1.10496896 0.98700742]
 [1.03285691 0.79267599]
 ...
 [1.41463415 1.323795  ]
 [1.41463415 1.41827208]
 [1.87182073 1.47266863]]
[1.04598819 1.04598819 0.91276645 ... 1.36921457 1.41645312 1.67224468]
('004ed441-24db-4839-8b5d-7465e4ea2a0a', '00544f67-c07c-4a07-9c17-a7aee51d8b96', '005685b2-3de4-469b-b673-8f937ee8196e', '0159954c-1bd2-47a6-9e6f-94f8d179b712', '017b6193-e0b7-4fb9-8312-97db38f8fb28', '01e0355c-e1a7-4958-a45e-8ab87107fb2b', '0222d5bf-deb0-4c33-8452-5bf0ef392671', '02389c9e-8821-4c54-a1a7-912f0b66dd95', '026064df-af76-4b9f-b7b0-99f5bb6175ad', '02b6250d-b2f6-4ea3-8e40-a37c7076b13b', '02c85901-cb63-4e92-86bb-04c4ea657515', '02f5bb4a-4c87-4d94-b957-5a9eeec981e8', '02fca132-7235-4643-95a3-2684e2fb0714', '0345af44-acb5-4338-86b3-20e51ac98258', '03fd95e1-6d36-4a1e-942b-d9adeb2750e5', '046c7976-ca14-4a6c-8a58-3e6e80b26bbf', '052e067c-0993-44ab-80b7-a69042330c58', '05a3b9dd-bbe4-4c4d-8

In [119]:
(1.0199858668253456+1.0371845444528067)/2

1.028585205639076